# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
with open(output_data_file,'r',encoding='utf-8') as csv:
    weather_read = pd.read_csv(csv,na_filter=False)
weather_read.describe(include='all')

#convert to readable format
weather_read["Date (Formatted)"] = pd.to_datetime(weather_read['Date'],unit='s')

#remove time
weather_read['Date (Formatted)'] = pd.to_datetime(weather_read['Date (Formatted)']).dt.date
weather_read['Date (Formatted)'] = pd.to_datetime(weather_read['Date (Formatted)']).dt.strftime('%m/%d/%Y')
weather_read.head()

,City,Country,Lat,Long,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date,Date (Formatted)
0,Bredasdorp,ZA,-34.5322,20.0403,61.41,52,81,8.75,1650652147,04/22/2022
1,Nikolskoye,RU,59.7035,30.7861,51.69,72,100,3.96,1650652147,04/22/2022
2,Arraial do Cabo,BR,-22.9661,-42.0278,82.26,65,0,9.22,1650652148,04/22/2022
3,Portland,US,45.5234,-122.6762,59.38,71,20,4.00,1650651962,04/22/2022
4,Aguazul,CO,5.1728,-72.5471,82.76,73,89,4.34,1650652148,04/22/2022


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [12]:
locations = weather_read[['Lat','Long']]
humidity = weather_read['Humidity (%)']

# Customize the size of the figure
figure_layout = {
    'width': '1200px',
    'height': '900px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=figure_layout)

In [13]:
heat_layer = gmaps.heatmap_layer(locations,weights = humidity,)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='900px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
#conditionals for the "Ideal" weather that is required
new_weather = weather_read.loc[(weather_read['Max Temp (F)'] > 70) & (weather_read['Max Temp (F)'] < 80) &
                               (weather_read['Wind Speed (mph)'] < 10) & (weather_read['Cloudiness (%)'] == 0)]
new_weather

,City,Country,Lat,Long,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date,Date (Formatted)
176,Ocampo,MX,21.6500,-101.5000,76.73,26,0,2.46,1650652208,04/22/2022
241,São Filipe,CV,14.8961,-24.4956,75.42,71,0,7.14,1650652230,04/22/2022
336,Hamilton,US,39.1834,-84.5333,76.98,46,0,9.22,1650652242,04/22/2022
369,Cabo San Lucas,MX,22.8909,-109.9124,79.90,69,0,8.05,1650652058,04/22/2022
464,Florianópolis,BR,-27.6146,-48.5012,78.62,72,0,8.05,1650652308,04/22/2022
503,Clarksburg,US,39.2807,-80.3445,73.42,39,0,3.44,1650652321,04/22/2022


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
hotel_df = new_weather
hotel_df["Hotel Name"] = ''

# Define the url required for the API
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {'radius':5000,
         'type':'hotel',
         'key':g_key}

hotel = []

#iterate through each row and get a hotel for each city
for index,row in hotel_df.iterrows():
    #set coordinate for row
    coord = f'{row["Lat"]},{row["Long"]}'
    params['location'] = coord
    
    #API request on google with json
    response_hotels = requests.get(url,params=params).json()

    #get results only
    results = response_hotels['results']
    hotel.append(results[0]['name'])

hotel_df["Hotel Name"] = hotel
hotel_df
    

C:\Users\KAIMUR~1\AppData\Local\Temp/ipykernel_5620/824793704.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ''
C:\Users\KAIMUR~1\AppData\Local\Temp/ipykernel_5620/824793704.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = hotel


,City,Country,Lat,Long,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date,Date (Formatted),Hotel Name
176,Ocampo,MX,21.6500,-101.5000,76.73,26,0,2.46,1650652208,04/22/2022,Ocampo
241,São Filipe,CV,14.8961,-24.4956,75.42,71,0,7.14,1650652230,04/22/2022,Sao Filipe
336,Hamilton,US,39.1834,-84.5333,76.98,46,0,9.22,1650652242,04/22/2022,Cincinnati
369,Cabo San Lucas,MX,22.8909,-109.9124,79.90,69,0,8.05,1650652058,04/22/2022,Cabo San Lucas
464,Florianópolis,BR,-27.6146,-48.5012,78.62,72,0,8.05,1650652308,04/22/2022,Florianópolis
503,Clarksburg,US,39.2807,-80.3445,73.42,39,0,3.44,1650652321,04/22/2022,Clarksburg


In [38]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [39]:
# Add marker layer ontop of heat map, include popup boxes with more information about each marker
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='900px', margin='0 auto 0 auto', padding='1px', wi…